<a href="https://colab.research.google.com/github/peterbmob/DHMVADoE/blob/main/Excercises/resp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Response surface model

In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from numpy.random import rand

Assume that you have found out that the temperature should be about 25C, pH about 8.0, and the amount of enzyme about 19 mg. Define an experimental domain with the centre point according to the values above (or your own best point, and with a range for each variable that is 10% of the full range given above

In [ ]:
def experiment(X):
  Xspan=[25, 4, 20]
  Xmin=[20, 5, 5]
  Xscale=[]
  for a, b,c in zip(X, Xmin,Xspan):
    Xscale.append((a-b)/c)

  koeff=[[0.643, -0.686, -0.341], [0.704, 0.704, -0.087], [0.3, -0.184, 0.936]]
  koeff=np.array(koeff)

  max1=[0.21, 0.77, 0.68]
  cent1=[]
  for a,b in zip(Xscale, max1):
    cent1.append(a-b)

  koord1=np.matmul(koeff.T, cent1)
  max2=[0.73, 0.42, 0.19]
  cent2=[]
  for a, b in zip(Xscale, max2):
    cent2.append(a-b)

  Resp1=8/(0.1+2*koord1[0]*koord1[0]+0.5*koord1[1]*koord1[1]+0.9*koord1[2]*koord1[2])
  Resp2=1/(0.05+10*(np.sum(np.square(cent2), axis=0)))
  Resp=(15+Resp1+Resp2)+0.1*(rand()-rand()+rand()-rand()+rand()-rand()+rand()-rand()+rand()-rand()+rand()-rand())
  return Resp


In [ ]:
# first try
#create list of data for high and low.
dat = [('T',20, 21.5, 23),
       ('p',5, 5.2, 5.4),
       ('A', 15, 15.8, 16.6)]

In [ ]:
# second try
max=1
#create list of data for high and low.
dat = [('T',March_real.iloc[max].Temperature-0.5, March_real.iloc[max].Temperature, March_real.iloc[max].Temperature+0.5),
        ('p', March_real.iloc[max].pH-0.5, March_real.iloc[max].pH,  March_real.iloc[max].pH+0.5),
        ('A',March_real.iloc[max].Amount-0.5, March_real.iloc[max].Amount, March_real.iloc[max].Amount+0.5 )]

In [ ]:
inputs_labels = {'T' : 'Temperature',
                 'p' : 'pH',
                 'A':'Amount'}



# create pandas dataframe in a pandas dataframe
inputs_df = pd.DataFrame(dat,columns=['index','low','center','high'])
inputs_df = inputs_df.set_index(['index'])
inputs_df['label'] = inputs_df.index.map( lambda z : inputs_labels[z] )

#print dataframe
inputs_df

In [ ]:
# compute averages and span
inputs_df['average'] = inputs_df.apply( lambda z : ( z['high'] + z['low'])/2 , axis=1)
inputs_df['span'] = inputs_df.apply( lambda z : ( z['high'] - z['low'])/2 , axis=1)

# encode the data
inputs_df['encoded_low'] = inputs_df.apply( lambda z : ( z['low']  - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_center'] = inputs_df.apply( lambda z : ( z['center'] - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_high'] = inputs_df.apply( lambda z : ( z['high'] - z['average'] )/( z['span'] ), axis=1)

inputs_df = inputs_df.drop(['average','span'],axis=1)

inputs_df

In [ ]:
import itertools
encoded_inputs= list(itertools.product([-1,1],[-1,1], [-1,1]))
encoded_inputs
for i in range(0,1):
    encoded_inputs.append((0,0,0))
encoded_inputs

In [ ]:
results=pd.DataFrame(encoded_inputs)
results=results[results.columns[::-1]]
results.columns=['T','p', 'A']
results

In [ ]:
def parse_values(x):
    if x < 2:
       return x * 10
    elif x < 4:
       return x ** 2
    else:
       return x + 10

real_experiment = results
var_labels = []
for var in ['T','p', 'A']:
    var_label = inputs_df.loc[var]['label']
    var_labels.append(var_label)
    real_experiment[var_label] = results.apply(
        lambda z : inputs_df.loc[var]['low'] if z[var]<0 else (inputs_df.loc[var]['high'] if z[var]>0 else inputs_df.loc[var]['center']),
        axis=1)



print("The values of each real variable in the experiment:")
real_experiment[var_labels]

In [ ]:
resp=[]
for i in range(len(real_experiment.index)):
   X=[real_experiment.Temperature[i], real_experiment.pH[i],real_experiment.Amount[i]]
   resp.append(experiment(X))

results['y']= resp
results

In [ ]:
results['y'].sort_values(ascending=False)
order=results['y'].sort_values(ascending=False).index
order

In [ ]:
y1 = results['y']
xlabs=['T','p','A']
x = results[xlabs]
x = sm.add_constant(x)

res1 = smf.ols(formula='y ~ T + p + A + T:p + T:A + A:p + T:p:A', data=results).fit()

res1.summary()

p-values are high for the interaction coefficients

In [ ]:
y1 = results['y']
xlabs=['T','p','A']
x = results[xlabs]
x = sm.add_constant(x)

res1 = smf.ols(formula='y ~ T + p + A', data=results).fit()

res1.summary()

This leaves with the first order model: ybar = 23.1511 + 0.6881T + 0.7681p +0.3016A  

So, now, for any t and T, we can predict y. This fits a flat surface and it tells us that the predicted y is a function of T, pH and A and the coefficients are the gradient of this function. We are working in coded variables, which means that the coefficients are unitless.

If we move 0.6881 in the direction of T and then 0.7681 and 0.3016 in the direction of pH and A, this will be the direction of steepest ascent. All we know is that this flat surface is one side of the "hill" forming our maxima.

With the method of steepest descent, we can now start marching up the hill taking additional measurements at each (T,pH, A) until the response starts to decrease. If we start at 0 (in coded units), then we can do series of single experiments on this path up the hill of the steepest ascent. If swe do this at a step size of t=1, then:



In [ ]:
Origin = [results['T'].iloc[order[0]], results['p'].iloc[order[0]], results['A'].iloc[order[0]]]
coeff= res1.params
delta=[coeff[1]/coeff[1],coeff[2]/coeff[1],coeff[3]/coeff[1] ]
marchT=[Origin[0]];marchp=[Origin[1]];marcha=[Origin[2]]

for i in range(0,4):
    marchT.append(Origin[0]+(i+1)*delta[0])
    marchp.append(Origin[1]+(i+1)*delta[1])
    marcha.append(Origin[1]+(i+1)*delta[2])

March=pd.DataFrame({'T':marchT, 'p':marchp, 'A':marcha})
ypred=res1.predict(March)

March['ypred']=ypred


# Lets do the real experiment
#assume we take steps in T

Origin_real=[results['Temperature'].iloc[order[0]], results['pH'].iloc[order[0]], results['Amount'].iloc[order[0]]]
marchT_real=[];marchp_real=[];marcha_real=[]
for i in range(len(March)):
    marchT_real.append(Origin_real[0]+(i+1)*delta[0])
    marchp_real.append(Origin_real[1]+(i+1)*delta[1])
    marcha_real.append(Origin_real[2]+(i+1)*delta[2])

March_real=pd.DataFrame({'Temperature':marchT_real, 'pH':marchp_real, 'Amount':marcha_real})

resp=[]
for i in range(len(March_real)):
   X=[March_real.Temperature[i], March_real.pH[i],March_real.Amount[i]]
   resp.append(experiment(X))

March['y_real']= resp
March



In [ ]:
March['ypred'].plot(marker='o')
March['y_real'].plot(marker='o')


In [ ]:
March['y_real'].sort_values(ascending=False)

In [ ]:
March_real.iloc[1]


When ready, use higher order model...


In [ ]:
res1 = smf.ols(formula='y ~ T + p + A + T:p + I(T**2) + I(p**2)', data=results).fit()
print(res1.summary())

In [ ]:
inputs_df #['low']

In [ ]:

np.random.uniform(inputs_df['low'][0],inputs_df['high'][0], 100)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

T = np.random.uniform(inputs_df['encoded_low'][0],inputs_df['encoded_high'][0], 1000)
p = np.random.uniform(inputs_df['encoded_low'][1],inputs_df['encoded_high'][1], 1000)
A = np.random.uniform(inputs_df['encoded_low'][2],inputs_df['encoded_high'][2], 1000)
data=pd.DataFrame({'T':T, 'p':p, 'A':A})

c = res1.predict(data)
data['ypred']=c
img = ax.scatter(T, p, A, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
data['ypred'].idxmax()
data.iloc[data['ypred'].idxmax()]